In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from nltk.tokenize import RegexpTokenizer
from nltk import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB

import time
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [28]:
df = pd.read_csv("dataset/final_data.csv")

In [29]:
df.head(20)

,Unnamed: 0,title,subreddit,author,subreddit_id,id,comments,combined_title_&_comment,subred_numeric,tokens,num_words,average_word_len,lems,stems,neg,neu,pos,compound,sent_score
0,0,Mulvaney acknowledges Ukraine aid was withheld...,Liberal,first52,t5_2qxt5,djdfhm,NaN,Mulvaney acknowledges Ukraine aid was withheld...,0,"['Mulvaney', 'acknowledges', 'Ukraine', 'aid',...",10,6.200000,Mulvaney acknowledges Ukraine aid wa withheld ...,mulvaney acknowledg ukrain aid wa withheld to ...,0.000,0.769,0.231,0.4019,1
1,1,"I have concerns about trump ""hosting"" the G7 a...",Liberal,djinnisequoia,t5_2qxt5,djdeoe,NaN,"I have concerns about trump ""hosting"" the G7 a...",0,"['I', 'have', 'concerns', 'about', 'trump', 'h...",14,4.357143,I have concern about trump hosting the G7 at D...,I have concern about trump host the G7 at dora...,0.000,1.000,0.000,0.0000,0
2,2,Swing district Democrats build on cash advanta...,Liberal,Bobsind,t5_2qxt5,djbsw2,NaN,Swing district Democrats build on cash advanta...,0,"['Swing', 'district', 'Democrats', 'build', 'o...",10,6.100000,Swing district Democrats build on cash advanta...,swing district democrat build on cash advantag...,0.000,0.818,0.182,0.2500,1
3,3,Climate crisis will not be discussed at G7 nex...,Liberal,spaceghoti,t5_2qxt5,djay3y,NaN,Climate crisis will not be discussed at G7 nex...,0,"['Climate', 'crisis', 'will', 'not', 'be', 'di...",13,4.615385,Climate crisis will not be discussed at G7 nex...,climat crisi will not be discuss at G7 next ye...,0.255,0.745,0.000,-0.6249,-1
4,4,California cannabis king linked to Ukrainian w...,Liberal,StardustSpinner,t5_2qxt5,djaxo7,Here is another story about the same character...,California cannabis king linked to Ukrainian w...,0,"['California', 'cannabis', 'king', 'linked', '...",68,5.000000,California cannabis king linked to Ukrainian w...,california cannabi king link to ukrainian who ...,0.000,1.000,0.000,0.0000,0
5,5,"BTRTN 2020 Vision: Warren Soars, Blows to Bern...",Liberal,hornet7777,t5_2qxt5,djah86,NaN,"BTRTN 2020 Vision: Warren Soars, Blows to Bern...",0,"['BTRTN', '2020', 'Vision', 'Warren', 'Soars',...",15,4.400000,BTRTN 2020 Vision Warren Soars Blows to Bernie...,btrtn 2020 vision warren soar blow to berni an...,0.000,0.867,0.133,0.2500,1
6,6,Moody’s analytics predicts trump will win 2020...,Liberal,anonymous197800,t5_2qxt5,djae92,NaN,Moody’s analytics predicts trump will win 2020...,0,"['Moody', 's', 'analytics', 'predicts', 'trump...",10,5.000000,Moody s analytics predicts trump will win 2020...,moodi s analyt predict trump will win 2020 in ...,0.000,0.678,0.322,0.5859,1
7,7,G7 summit will be held at Trump's Doral resort...,Liberal,Philo1927,t5_2qxt5,dj956r,NaN,G7 summit will be held at Trump's Doral resort...,0,"['G7', 'summit', 'will', 'be', 'held', 'at', '...",15,4.000000,G7 summit will be held at Trump s Doral resort...,G7 summit will be held at trump s doral resort...,0.000,1.000,0.000,0.0000,0
8,8,Patriotism and Diversity - A New Flag?,Liberal,gerinteed,t5_2qxt5,dj5ttx,NaN,Patriotism and Diversity - A New Flag?,0,"['Patriotism', 'and', 'Diversity', 'A', 'New',...",6,5.000000,Patriotism and Diversity A New Flag,patriot and divers A new flag,0.000,1.000,0.000,0.0000,0
9,9,"Bush, Obama and all former presidents should s...",Liberal,Jeffersonpaine2016,t5_2qxt5,dj4wvp,Uhhh... They all already have... Fuck Bush. He...,"Bush, Obama and all former presidents should s...",0,"['Bush', 'Obama', 'and', 'all', 'former', 'pre...",103,4.300971,Bush Obama and all former president should spe...,bush obama and all former presid should speak ...,0.000,1.000,0.000,0.0000,0


In [30]:
df['lems']

0       Mulvaney acknowledges Ukraine aid wa withheld ...
1       I have concern about trump hosting the G7 at D...
2       Swing district Democrats build on cash advanta...
3       Climate crisis will not be discussed at G7 nex...
4       California cannabis king linked to Ukrainian w...
                              ...                        
1995    Mr Trump choose your word wisely backstabbing ...
1996    The sibling of an ARMED robber who wa shot and...
1997                  Remind me again how Trump is racist
1998    North Korea warns UN it could reverse crucial ...
1999    Soros Mole Fiona Hill Is Second White House Wh...
Name: lems, Length: 2000, dtype: object

## Cross Validation for the Lems

In [45]:
v = TfidfVectorizer(decode_error = 'replace', encoding = 'utf-8')
X = v.fit_transform(df['lems'].values.astype(str))

### Variable Creation 

In [46]:
X = df['lems']
y = df['subred_numeric']

### Train test split 

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [47]:
X_train_trans = v.fit_transform(X_train.values.astype(str))

In [50]:
y_train_trans = v.transform(y_train.values.astype(str))

In [33]:
pipe_1 = Pipeline([
    ('cvec', CountVectorizer()),
    ('logreg', LogisticRegression())])

pipe_2 = Pipeline([
    ('tvec', TfidfVectorizer()),
    ('logreg', LogisticRegression())])

pipe_3 = Pipeline ([
    ('cvec', CountVectorizer()),
    ('knn', KNeighborsClassifier())])

pipe_4 = Pipeline([
    ('tvec', TfidfVectorizer()),
    ('knn', KNeighborsClassifier())])

pipe_5 = Pipeline([
    ('cvec', CountVectorizer()),
    ('dectree', DecisionTreeClassifier())])

pipe_6 = Pipeline([
    ('cvec', CountVectorizer()),
    ('bag', BaggingClassifier())])

pipe_7 = Pipeline([
    ('tvec', TfidfVectorizer()),
    ('dectree', DecisionTreeClassifier())])

pipe_8 = Pipeline([
    ('tvec', TfidfVectorizer()),
    ('bag', BaggingClassifier())])

pipe_9 = Pipeline([
    ('cvec', CountVectorizer()),
    ('nb' , MultinomialNB())])

pipe_10 = Pipeline([
    ('cvec', CountVectorizer()),
    ('gaus', GaussianNB())])

pipe_11 = Pipeline([
    ('tvec', TfidfVectorizer()),
    ('nb' , MultinomialNB())])

pipe_12 = Pipeline([
    ('tvec', TfidfVectorizer()),
    ('gaus', GaussianNB())])



In [34]:
pipe_1_params = {
    'cvec__stop_words' : ['english', None],
    'cvec__max_features' : [2500, 3000, 3500],
    'cvec__min_df': [1,2],
    'cvec__max_df': [200, 300],
    'cvec__ngram_range': ([1,1], (1,2), (1,3)),
    'logreg__C' :[0.01, 0.1, 1, 10],
    'logreg__random_state' : [40, 50, 100]
}

pipe_2_params = {
    'tvec__stop_words' : ['english', None],
    'tvec__max_features' : [2500, 3000, 3500],
    'tvec__min_df': [1,2],
    'tvec__max_df': [200, 300],
    'tvec__ngram_range': ([1,1], (1,2), (1,3)),
    'logreg__C' :[0.01, 0.1, 1, 10],
    'logreg__random_state' : [40, 50, 100]
}
#unreliable 
pipe_3_params = {
    'cvec__stop_words' : ['english', None],
    'cvec__max_features' : [2500, 3000, 3500],
    'cvec__min_df': [1,2],
    'cvec__max_df': [200, 300],
    'cvec__ngram_range': ([1,1], (1,2), (1,3)),
    'knn__n_neighbors': [1,5,10]
}
#unreliable 
pipe_4_params = {
    'tvec__stop_words' : ['english', None],
    'tvec__max_features' : [2500, 3000, 3500],
    'tvec__min_df': [1,2],
    'tvec__max_df': [200, 300],
    'tvec__ngram_range': ([1,1], (1,2), (1,3)),
    'knn__n_neighbors': [1,5,10]
}

#unreliable 
pipe_5_params = {
    'cvec__stop_words' : ['english', None],
    'cvec__max_features' : [2500, 3000, 3500],
    'cvec__min_df': [1,2],
    'cvec__max_df': [200, 300],
    'cvec__ngram_range': ([1,1], (1,2), (1,3)),
    'dectree__max_depth': [2,5,7, 10],
    'dectree__max_features': ['auto', 'sqrt'],
    'dectree__min_samples_leaf': [1,2,4],
    'dectree__min_samples_split': [2,5,10]   
}

pipe_6_params = {'cvec__stop_words' : ['english', None],
    'cvec__max_features' : [2500, 3000, 3500],
    'cvec__min_df': [1,2],
    'cvec__max_df': [200, 300],
    'cvec__ngram_range': ([1,1], (1,2), (1,3)),
    'bag__max_features' : [100, 200],
    'bag__bootstrap': [True, False]
    }
# unreliable 
pipe_7_params = {
    'tvec__stop_words' : ['english', None],
    'tvec__max_features' : [2500, 3000, 3500],
    'tvec__min_df': [1,2],
    'tvec__max_df': [200, 300],
    'tvec__ngram_range': ([1,1], (1,2), (1,3)),
    'dectree__max_depth': [2,5,7, 10],
    'dectree__max_features': ['auto', 'sqrt'],
    'dectree__min_samples_leaf': [1,2,4, 7],
    'dectree__min_samples_split': [2,5,10]  }

pipe_8_params = {
    'tvec__stop_words' : ['english', None],
    'tvec__max_features' : [2500, 3000, 3500],
    'tvec__min_df': [1,2],
    'tvec__max_df': [200, 300],
    'tvec__ngram_range': ([1,1], (1,2), (1,3)),
    'bag__max_features' : [100, 200],
    'bag__bootstrap': [True, False]}

pipe_9_params = {'cvec__stop_words' : ['english', None],
    'cvec__max_features' : [2500, 3000, 3500],
    'cvec__min_df': [1,2],
    'cvec__max_df': [200, 300],
    'cvec__ngram_range': ([1,1], (1,2), (1,3)),
    "nb__class_prior" : [None, True],
    "nb__fit_prior": [None, True]
                }
#don't work 
pipe_10_params ={
    'cvec__stop_words' : ['english', None],
    'cvec__max_features' : [2500, 3000, 3500],
    'cvec__min_df': [1,2],
    'cvec__max_df': [200, 300],
    'cvec__ngram_range': ([1,1], (1,2), (1,3))}

pipe_11_params = {
    'tvec__stop_words' : ['english', None],
    'tvec__max_features' : [2500, 3000, 3500],
    'tvec__min_df': [1,2],
    'tvec__max_df': [200, 300],
    'tvec__ngram_range': ([1,1], (1,2), (1,3)),
    #"nb__class_prior" : [None, True],
    #"nb__fit_prior": [None, True]
}

#don't work 
pipe_12_params = {'tvec__stop_words' : ['english', None],
    'tvec__max_features' : [2500, 3000, 3500],
    'tvec__min_df': [1,2],
    'tvec__max_df': [200, 300],
    'tvec__ngram_range': ([1,1], (1,2), (1,3))}

    

# Gridsearch Time!

## Gridsearch 1- CountVectorizer and LogReg

In [35]:
grid_1 = GridSearchCV(pipe_1, pipe_1_params, cv =5, verbose =1  )

In [51]:

grid_1.fit(X_train_trans, y_train_trans)

Fitting 5 folds for each of 864 candidates, totalling 4320 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


AttributeError: lower not found

In [32]:
print(grid_1.best_score_)

0.6846057571964956


In [33]:
print(grid_1.best_estimator_)

Pipeline(memory=None,
     steps=[('cvec', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=200, max_features=2500, min_df=1,
        ngram_range=(1, 3), preprocessor=None, stop_words='english',
        ..., penalty='l2', random_state=40, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])


## Gridsearch 2- TFIDIF and LogReg

In [37]:
grid_2 = GridSearchCV(pipe_2, pipe_2_params, cv = 5, verbose =1 )

In [38]:
t0 = time.time()

In [39]:
grid_2.fit(X_train, y_train)


Fitting 5 folds for each of 864 candidates, totalling 4320 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


ValueError: np.nan is an invalid document, expected byte or unicode string.

In [43]:
print(grid_2.best_score_)

0.6933667083854819


In [42]:
print(grid_2.best_estimator_)

Pipeline(memory=None,
     steps=[('tvec', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=200, max_features=3500, min_df=1,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
..., penalty='l2', random_state=40, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])


## Gridsearch 3: CountVectorizer and KNN 

In [40]:
grid_3 = GridSearchCV(pipe_3, pipe_3_params, cv = 5, verbose =1 )

In [41]:
grid_3.fit(X_train, y_train)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1080 out of 1080 | elapsed:  1.0min finished
/Users/nathan/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('cvec', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ki',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'cvec__stop_words': ['english', None], 'cvec__max_features': [2500, 3000, 3500], 'cvec__min_df': [1, 2], 'cvec__max_df': [200, 300], 'cvec__ngram_range': ([1, 1], (1, 2), (1, 3)), 'knn__n_neighbors': [1, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [44]:
print(grid_3.best_score_)

0.6533166458072591


In [45]:
print(grid_3.best_estimator_)

Pipeline(memory=None,
     steps=[('cvec', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=200, max_features=2500, min_df=2,
        ngram_range=(1, 3), preprocessor=None, stop_words=None,
        strip...ki',
           metric_params=None, n_jobs=None, n_neighbors=1, p=2,
           weights='uniform'))])


## Gridsearch: 4 TFIDIF and KNN

In [47]:
grid_4 = GridSearchCV(pipe_4, pipe_4_params, cv = 5, verbose =1 )

In [48]:
grid_4.fit(X_train, y_train)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1080 out of 1080 | elapsed:  1.1min finished
/Users/nathan/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('tvec', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...ki',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'tvec__stop_words': ['english', None], 'tvec__max_features': [2500, 3000, 3500], 'tvec__min_df': [1, 2], 'tvec__max_df': [200, 300], 'tvec__ngram_range': ([1, 1], (1, 2), (1, 3)), 'knn__n_neighbors': [1, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [50]:
print(grid_4.best_score_)

0.6683354192740926


In [51]:
print(grid_4.best_estimator_)

Pipeline(memory=None,
     steps=[('tvec', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=300, max_features=3000, min_df=1,
        ngram_range=[1, 1], norm='l2', preprocessor=None, smooth_idf=True,
...ki',
           metric_params=None, n_jobs=None, n_neighbors=1, p=2,
           weights='uniform'))])


## Gridseach: 5 CountVectorizer and DecisionTree 

In [54]:
grid_5 = GridSearchCV(pipe_5, pipe_5_params, cv = 5, verbose =1 )

In [55]:
grid_5.fit(X_train, y_train)

Fitting 5 folds for each of 5184 candidates, totalling 25920 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 25920 out of 25920 | elapsed: 14.7min finished
/Users/nathan/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('cvec', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...      min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'cvec__stop_words': ['english', None], 'cvec__max_features': [2500, 3000, 3500], 'cvec__min_df': [1, 2], 'cvec__max_df': [200, 300], 'cvec__ngram_range': ([1, 1], (1, 2), (1, 3)), 'dectree__max_depth': [2, 5, 7, 10], 'dectree__max_features': ['auto', 'sqrt'], 'dectree__min_samples_leaf': [1, 2, 4], 'dectree__min_samples_split': [2, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=N

In [58]:
print(grid_5.best_score_)

0.5819774718397998


In [59]:
print(grid_5.best_estimator_)

Pipeline(memory=None,
     steps=[('cvec', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=300, max_features=3500, min_df=2,
        ngram_range=[1, 1], preprocessor=None, stop_words=None,
        strip...      min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'))])


## GridSearch: 6 CountVectorizer and Bagging 

In [40]:
grid_6 = GridSearchCV(pipe_6, pipe_6_params, cv = 5, verbose =1 )

In [41]:
grid_6.fit(X_train, y_train)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


ValueError: np.nan is an invalid document, expected byte or unicode string.

In [65]:
print(grid_6.best_score_)

0.6871088861076345


In [66]:
print(grid_6.best_estimator_)

Pipeline(memory=None,
     steps=[('cvec', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=200, max_features=2500, min_df=1,
        ngram_range=[1, 1], preprocessor=None, stop_words='english',
        ...imators=10, n_jobs=None, oob_score=False, random_state=None,
         verbose=0, warm_start=False))])


## Gridsearch: 7 TFIDIF and DecisionTree

In [67]:
grid_7 = GridSearchCV(pipe_7, pipe_7_params, cv = 5, verbose =1 )

In [68]:
grid_7.fit(X_train, y_train)

Fitting 5 folds for each of 6912 candidates, totalling 34560 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 34560 out of 34560 | elapsed: 19.5min finished
/Users/nathan/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('tvec', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...      min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'tvec__stop_words': ['english', None], 'tvec__max_features': [2500, 3000, 3500], 'tvec__min_df': [1, 2], 'tvec__max_df': [200, 300], 'tvec__ngram_range': ([1, 1], (1, 2), (1, 3)), 'dectree__max_depth': [2, 5, 7, 10], 'dectree__max_features': ['auto', 'sqrt'], 'dectree__min_samples_leaf': [1, 2, 4, 7], 'dectree__min_samples_split': [2, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scorin

In [69]:
print(grid_7.best_score_)

0.590738423028786


In [70]:
print(grid_7.best_estimator_)

Pipeline(memory=None,
     steps=[('tvec', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=300, max_features=3000, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
...      min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'))])


## Gridsearch: 8 TFIDF and Bagging 

In [71]:
grid_8 = GridSearchCV(pipe_8, pipe_8_params, cv = 5, verbose =1 )

In [72]:
grid_8.fit(X_train, y_train)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1440 out of 1440 | elapsed:  1.8min finished
/Users/nathan/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('tvec', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...imators=10, n_jobs=None, oob_score=False, random_state=None,
         verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'tvec__stop_words': ['english', None], 'tvec__max_features': [2500, 3000, 3500], 'tvec__min_df': [1, 2], 'tvec__max_df': [200, 300], 'tvec__ngram_range': ([1, 1], (1, 2), (1, 3)), 'bag__max_features': [100, 200], 'bag__bootstrap': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [73]:
print(grid_8.best_score_)

0.6821026282853567


In [74]:
print(grid_8.best_estimator_)

Pipeline(memory=None,
     steps=[('tvec', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=200, max_features=3500, min_df=2,
        ngram_range=[1, 1], norm='l2', preprocessor=None, smooth_idf=True,
...imators=10, n_jobs=None, oob_score=False, random_state=None,
         verbose=0, warm_start=False))])


## Gridsearch: 9 CountVectorizer and MultinomialNB

In [78]:
grid_9 = GridSearchCV(pipe_9, pipe_9_params, cv = 5, verbose =1 )

In [79]:
grid_9.fit(X_train, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 360 out of 360 | elapsed:   12.8s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('cvec', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)), ('nb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'cvec__stop_words': ['english', None], 'cvec__max_features': [2500, 3000, 3500], 'cvec__min_df': [1, 2], 'cvec__max_df': [200, 300], 'cvec__ngram_range': ([1, 1], (1, 2), (1, 3))},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [80]:
print(grid_9.best_score_)

0.7021276595744681


In [81]:
print(grid_9.best_estimator_)

Pipeline(memory=None,
     steps=[('cvec', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=200, max_features=2500, min_df=1,
        ngram_range=[1, 1], preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)), ('nb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])


In [114]:
grid_9.best_estimator_

Pipeline(memory=None,
     steps=[('cvec', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=200, max_features=2500, min_df=1,
        ngram_range=[1, 1], preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)), ('nb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [116]:
cv_test = CountVectorizer(max_df=200,
                          max_features=2500,
                          min_df=1,
                          ngram_range=[1, 1],
                          stop_words='english')


In [117]:
X_train_cvec = cv_test.fit_transform(X_train)

In [118]:
nb_test = MultinomialNB(alpha=1.0)

In [119]:
nb_test.fit(X_train_cvec, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [132]:
feature_df = pd.DataFrame(np.exp(nb_test.feature_log_prob_), columns = cv_test.get_feature_names()).T

In [133]:
feature_df.head()

,0,1
000,0.000535,0.000372
10,0.000535,0.000372
11,0.000357,0.000372
110,0.000178,0.000372
11th,0.000535,0.000186


In [141]:

feature_df.sort_values(by=0, ascending=False)

,0,1
trump,0.023177,0.012095
biden,0.003744,0.004280
impeachment,0.003566,0.003722
ha,0.003566,0.001303
conservative,0.003387,0.001116
president,0.003209,0.002977
house,0.003031,0.000744
ukraine,0.003031,0.001303
new,0.002674,0.000930
state,0.002496,0.000930


## Gridsearch: 10  CountVectorizer and GaussianNB

In [11]:
#Don't run 
grid_10 = GridSearchCV(pipe_10, pipe_10_params, cv = 5, verbose =1 )

In [12]:
grid_10.fit(X_train.todense(), y_train)

AttributeError: 'Series' object has no attribute 'todense'

In [ ]:
print(grid_10.best_score_)

In [ ]:
print(grid_10.best_estimator_)

## Gridsearch: 11 TFIDF and MultinominalNB

In [102]:
grid_11 = GridSearchCV(pipe_11, pipe_11_params, cv = 5, verbose =1 )

In [103]:
grid_11.fit(X_train, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 360 out of 360 | elapsed:   12.6s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('tvec', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...True,
        vocabulary=None)), ('nb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'tvec__stop_words': ['english', None], 'tvec__max_features': [2500, 3000, 3500], 'tvec__min_df': [1, 2], 'tvec__max_df': [200, 300], 'tvec__ngram_range': ([1, 1], (1, 2), (1, 3))},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [106]:
print(grid_11.best_score_)

0.6971214017521903


In [107]:
print(grid_11.best_estimator_)

Pipeline(memory=None,
     steps=[('tvec', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=200, max_features=2500, min_df=1,
        ngram_range=[1, 1], norm='l2', preprocessor=None, smooth_idf=True,
...True,
        vocabulary=None)), ('nb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])


## Gridsearch: 12 TFIDF and GaussianNB

In [104]:
#Gaussian not working 
grid_12 = GridSearchCV(pipe_12, pipe_12_params, cv = 5, verbose =1 )

In [108]:
grid_12.fit(X_train, y_train)

In [109]:
print(grid_12.best_score_)

In [110]:
print(grid_12.best_estimator_)